In [1]:
import numpy as np
import pandas as pd
import math


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_pickle('/content/drive/MyDrive/neuronske/data/full_sequences.pkl')

In [ ]:
training = data[data['event_label'] == 'Alarm_bell_ringing']

X = np.array(training.mfccs.tolist())
y = np.array(training.combine_squence_per_filename_and_event.tolist())

X = np.einsum('ijk->ikj', X)

# split the dataset 
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [ ]:
X_train.shape

(313, 431, 13)

In [4]:
# USED ONLY FOR TESTING
# NOT GOOD FOR SEQ TO SEQ, IT IS NOT JUST CLASSIFICATION
# IMPLEMENT WHAT THEY DID IN TASK WHERE WE GOT THE IDEA

# CONCLUSIONS: TOO MUCH FALSE NEGATIVES - BAD RECALL
#              BIGGER LATENT_TIM -> BETTER RESULTS -- LAT_DIM-F1 : 13-68, 64-70 256-73
def simple_evaluation(y_pred, y_test):
  tp = 0
  tn = 0
  fp = 0
  fn = 0

  for i, pred_arr in enumerate(y_pred):
    for j, pred_value in enumerate(pred_arr):
      predicted = pred_value
      real = y_test[i][j]
      if predicted and real:
        tp = tp + 1
      elif not(predicted) and real:
        fn = fn + 1
      elif predicted and not(real):
        fp = fp + 1
      elif not(predicted) and not(real):
        tn = tn + 1
    # break

  precision = tp / (tp + fp)
  recall = tp / (tp + fn)
  f1 = 2 * precision * recall / (precision + recall)

  print("PRECISION: " + str(precision))
  print("RECALL: " + str(recall))
  print("F1: " + str(f1))
  print("-"*20)
  print("tp: " + str(tp))
  print("tn: " + str(tn))
  print("fp: " + str(fp))
  print("fn: " + str(fn))


In [5]:
def chunks(lst, n):
  """Yield successive n-sized chunks from lst."""
  return ([lst[i:i + n] for i in range(0, len(lst), n)])


def split_sequences_into_seconds(X, y, seconds = 1):
  sequence_len = X.shape[1]
  ticks_per_chosen_second = math.ceil(sequence_len / (10 / seconds))
  num_of_parts_of_sequence = math.ceil(sequence_len / ticks_per_chosen_second)
  # ticks_per_sequence = math.ceil(sequence_len / num_of_parts_of_sequence)

  # print(ticks_per_chosen_second)
  # print(num_of_parts_of_sequence)
  # print(ticks_per_sequence)

  X_new = []
  y_new = []
  for i, sequence in enumerate(X):
    X_splits = chunks(sequence, ticks_per_chosen_second)
    y_splits = chunks(y[i], ticks_per_chosen_second)
    x_false_instances = np.array([]).reshape(0,X_splits[0].shape[1])
    for j, X_split in enumerate(X_splits):
      X_temp = X_split
      y_temp = y_splits[j]
      if len(x_false_instances) < ticks_per_chosen_second:
        false_indices = np.where(~y_temp)
        x_false_instances = np.append(x_false_instances, X_split[false_indices], axis=0)

      diff = ticks_per_chosen_second - len(X_temp)
      if diff != 0:
        y_temp = np.append(y_temp, [0] * diff)
        X_temp = np.append(X_temp, x_false_instances[0: diff], axis=0)
        
      X_new.append(X_temp)
      y_new.append(y_temp)


  return np.asarray(X_new), np.asarray(y_new)

In [ ]:
X_splitted, y_splitted = split_sequences_into_seconds(X, y, 0.2)
print(X.shape)
print(X_splitted.shape)
print(y.shape)
print(y_splitted.shape)

(392, 431, 13)
(18816, 9, 13)
(392, 431)
(18816, 9)


In [ ]:
# split the dataset 
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)
X_train, y_train = split_sequences_into_seconds(X_train, y_train, 0.2)
X_test, y_test = split_sequences_into_seconds(X_test, y_test, 0.2)

In [6]:
import keras
def build_model_cnn(input_shape, output_shape):
    """Generates CNN model
    :param input_shape (tuple): Shape of input set
    :return model: CNN model
    """

    # build network topology
    model = keras.Sequential()

    # 1st conv layer
    model.add(keras.layers.Conv2D(128, (3, 3), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # 2nd conv layer
    model.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # flatten output and feed it into dense layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(256, activation='relu'))
    model.add(keras.layers.Dropout(0.3))

    # output layer
    # output layer
    model.add(keras.layers.Dense(output_shape, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')

    return model

In [ ]:
input_shape = (X_train.shape[1], X_train.shape[2], 1)
output_shape = X_train.shape[1]
model = build_model_cnn(input_shape, output_shape)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 7, 11, 128)        1280      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 4, 6, 128)         0         
_________________________________________________________________
batch_normalization (BatchNo (None, 4, 6, 128)         512       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 2, 4, 128)         147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 2, 128)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 1, 2, 128)         512       
_________________________________________________________________
flatten (Flatten)            (None, 256)               0

In [ ]:
X_train.shape
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

In [ ]:
X_test.shape

(3792, 9, 13, 1)

In [7]:
import tensorflow.keras as keras
import matplotlib.pyplot as plt

def plot_history(history):
    """Plots accuracy/loss for training/validation set as a function of the epochs
        :param history: Training history of model
        :return:
    """

    fig, axs = plt.subplots(2)

    # create accuracy sublpot
    axs[0].plot(history.history["accuracy"], label="train accuracy")
    axs[0].plot(history.history["val_accuracy"], label="test accuracy")
    axs[0].set_ylabel("Accuracy")
    axs[0].legend(loc="lower right")
    axs[0].set_title("Accuracy eval")

    # create error sublpot
    axs[1].plot(history.history["loss"], label="train error")
    axs[1].plot(history.history["val_loss"], label="test error")
    axs[1].set_ylabel("Error")
    axs[1].set_xlabel("Epoch")
    axs[1].legend(loc="upper right")
    axs[1].set_title("Error eval")

    plt.show()

In [ ]:
# train model

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=64, epochs=110)

# plot accuracy/error for training and validation
# plot_history(history)

Epoch 1/110
235/235 [==============================] - 33s 59ms/step - loss: 0.3228 - accuracy: 0.1397 - val_loss: 0.3245 - val_accuracy: 0.0285
Epoch 2/110
235/235 [==============================] - 13s 56ms/step - loss: 0.2318 - accuracy: 0.1678 - val_loss: 0.2838 - val_accuracy: 0.2919
Epoch 3/110
235/235 [==============================] - 13s 57ms/step - loss: 0.2039 - accuracy: 0.1773 - val_loss: 0.2962 - val_accuracy: 0.3199
Epoch 4/110
235/235 [==============================] - 13s 54ms/step - loss: 0.1939 - accuracy: 0.2138 - val_loss: 0.2866 - val_accuracy: 0.1498
Epoch 5/110
235/235 [==============================] - 13s 57ms/step - loss: 0.1791 - accuracy: 0.2358 - val_loss: 0.2943 - val_accuracy: 0.2207
Epoch 6/110
235/235 [==============================] - 14s 58ms/step - loss: 0.1653 - accuracy: 0.2736 - val_loss: 0.2757 - val_accuracy: 0.2782
Epoch 7/110
144/235 [=================>............] - ETA: 4s - loss: 0.1640 - accuracy: 0.2827

KeyboardInterrupt: ignored

In [ ]:

y_pred = model.predict(X_test, batch_size=64, verbose=1)
y_pred = (y_pred > 0.5)

In [ ]:
num = 1
print(y_pred[num])
print(y_test[num])
print(y_test.shape)
print(y[num])

[False False False False False False False False False]
[0 0 0 0 0 0 0 0 0]
(3764, 9)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [ ]:
y_pred.shape[0]

3792

In [ ]:
y.shape

(392, 431)

In [8]:
# Doterati malo ne treba ovoliko parametara
def combine_predicted_to_original(y_pred, seconds, sequence_len):
  ticks_per_chosen_second = math.ceil(sequence_len / (10 / seconds))
  num_of_parts_of_sequence = math.ceil(sequence_len / ticks_per_chosen_second)
  y_pred_combined = []
  # Num of arrays at the end
  final_size = math.floor(y_pred.shape[0] / num_of_parts_of_sequence)

  print(num_of_parts_of_sequence)
  print(final_size)

  for i in range(0, final_size):
    current_arr = np.array([])
    for j in range(0, num_of_parts_of_sequence):
      current_arr = np.concatenate((current_arr, y_pred[i * num_of_parts_of_sequence + j])).astype(int)
    y_pred_combined.append(current_arr)
  return y_pred_combined

In [ ]:
combined_pred_y = combine_predicted_to_original(y, y_pred, 0.2, 431)
combined_test_y = combine_predicted_to_original(y, y_test, 0.2, 431)


48
79
48
79


In [ ]:
simple_evaluation(combined_pred_y, combined_test_y)

PRECISION: 0.8429460232049772
RECALL: 0.6703664081305162
F1: 0.7468156424581005
--------------------
tp: 5013
tn: 25716
fp: 934
fn: 2465


**ANSAMBLE**

In [9]:
data = pd.read_pickle('/content/drive/MyDrive/neuronske/data/full_sequences.pkl')
data.head()

,filename,mfccs,event_label,combine_squence_per_filename_and_event
0,2282.wav,"[[-659.70654, -618.7009, -581.4676, -577.1263,...",Dog,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
0,2282.wav,"[[-659.70654, -618.7009, -581.4676, -577.1263,...",Speech,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2719.wav,"[[-584.7399, -515.89514, -473.47324, -464.3819...",Electric_shaver_toothbrush,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2719.wav,"[[-584.7399, -515.89514, -473.47324, -464.3819...",Speech,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,1238.wav,"[[-401.96198, -400.9539, -411.34525, -405.876,...",Blender,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
# X_splitted, y_splitted = split_sequences_into_seconds(X, y, 0.2)

# print(X.shape)
# print(X_splitted.shape)
# print(y.shape)
# print(y_splitted.shape)

In [10]:
unique_events = np.sort(data['event_label'].unique())
print(unique_events)

['Alarm_bell_ringing' 'Blender' 'Cat' 'Dishes' 'Dog'
 'Electric_shaver_toothbrush' 'Frying' 'Running_water' 'Speech'
 'Vacuum_cleaner']


In [11]:
from sklearn.model_selection import train_test_split 

for index, event in enumerate(unique_events):
  training = data[data['event_label'] == event]

  X = np.array(training.mfccs.tolist())
  y = np.array(training.combine_squence_per_filename_and_event.tolist())

  X = np.einsum('ijk->ikj', X)

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)
  X_train, y_train = split_sequences_into_seconds(X_train, y_train, 5)
  X_test, y_test = split_sequences_into_seconds(X_test, y_test, 5)

  print("-"*80)
  print(str(index) + " ------ " + event + " " + str(len(X)))

  input_shape = (X_train.shape[1], X_train.shape[2], 1)
  output_shape = X_train.shape[1]
  print("X_train shape: ")
  print(X_train.shape)
  
  X_train = np.expand_dims(X_train, axis=-1)
  X_test = np.expand_dims(X_test, axis=-1)  
  model = build_model_cnn(input_shape, output_shape)
  model.summary()
  model.fit(X_train, y_train, validation_split=0.2, batch_size=128, epochs=120)
  model.save('/content/drive/MyDrive/neuronske/models/' + 'CNN_splitted_seq' + str(event) + '.h5')
  print("-"*80)


--------------------------------------------------------------------------------
0 ------ Alarm_bell_ringing 392
X_train shape: 
(626, 216, 13)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 214, 11, 128)      1280      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 107, 6, 128)       0         
_________________________________________________________________
batch_normalization (BatchNo (None, 107, 6, 128)       512       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 105, 4, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 53, 2, 128)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 53, 2, 1

In [15]:
# RESULTS LOOK OK
# ANALYZE HOW IT BEHAVES WITH OVERLAPING EVENTS
combined_references = []
combined_estimations = []

for index, event in enumerate(unique_events):
  training = data[data['event_label'] == event]

  X = np.array(training.mfccs.tolist())
  y = np.array(training.combine_squence_per_filename_and_event.tolist())

  X = np.einsum('ijk->ikj', X)

  indices = np.arange(X.shape[0])
  X_train, X_test, y_train, y_test, I_train, I_test = train_test_split(X, y, indices, test_size=0.2, random_state = 42)

  X_train, y_train = split_sequences_into_seconds(X_train, y_train, 5)
  X_test, y_test = split_sequences_into_seconds(X_test, y_test, 5)
  X_train = np.expand_dims(X_train, axis=-1)
  X_test = np.expand_dims(X_test, axis=-1) 

  print("-"*80)
  print(str(index) + " ------ " + event + " " + str(len(X)))
  model = keras.models.load_model('/content/drive/MyDrive/neuronske/models/' + 'CNN_splitted_seq' + str(event) + '.h5', compile=True)
  y_pred = model.predict(X_test, batch_size=64, verbose=1)
  y_pred = (y_pred > 0.5)
  simple_evaluation(y_pred, y_test)
  print("-"*80)

  combined_y_pred = combine_predicted_to_original(y_pred, 5, 431)
  combined_y_test = combine_predicted_to_original(y_test, 5, 431)

  simple_evaluation(combined_y_pred, combined_y_test)
  print("-"*80)

  references, estimations = prepare_predicted_sequences_for_evaluation_and_evaluate(event, combined_y_pred, I_test, training, events, 1)
  combined_references.extend(references)
  combined_estimations.extend(estimations)


print("-"*80)
print("-"*80)

print("FINAL EVALUATION: ")
print("-"*80)
evaluate_and_print_results(combined_references, combined_estimations)
print("-"*80)

--------------------------------------------------------------------------------
0 ------ Alarm_bell_ringing 392
3/3 [==============================] - 0s 5ms/step
PRECISION: 0.7817971906869348
RECALL: 0.543327092805563
F1: 0.641104536489152
--------------------
tp: 4063
tn: 25516
fp: 1134
fn: 3415
--------------------------------------------------------------------------------
2
79
2
79
PRECISION: 0.7817971906869348
RECALL: 0.543327092805563
F1: 0.641104536489152
--------------------
tp: 4063
tn: 25516
fp: 1134
fn: 3415
--------------------------------------------------------------------------------
Accuracy: 0.7937219730941704
Segment based metrics
  Evaluated length                  : 642.91 sec
  Evaluated files                   : 79 
  Segment length                    : 1.00 sec

  Overall metrics (micro-average)
  F-measure
    F-measure (F1)                  : 76.37 %
    Precision                       : 79.64 %
    Recall                          : 73.36 %
  Error rate
    E

In [14]:
events = pd.read_csv("/content/drive/MyDrive/neuronske/data/Synthetic_dataset/synthetic_dataset.csv", delimiter="\t") 
events.head()

,filename,onset,offset,event_label
0,10.wav,0.287159,1.057712,Alarm_bell_ringing
1,10.wav,1.717124,1.967124,Alarm_bell_ringing
2,10.wav,5.049573,6.013632,Alarm_bell_ringing
3,10.wav,7.240901,8.492352,Speech
4,1000.wav,4.352958,4.664092,Alarm_bell_ringing


In [13]:
!pip install sed_eval

import sed_eval
import dcase_util

def return_onsets_and_offsets_for_sequence(sequence, duration_seconds=10):
  single_tick_in_seconds = duration_seconds / len(sequence)
  onsets = []
  offsets = []

  error_allowance = 5
  error_counter = 0
  in_positive_sequence = 0
  
  for index, tick in enumerate(sequence):
    if tick and not(in_positive_sequence):
      in_positive_sequence = 1
      onsets.append(index * single_tick_in_seconds)
    elif not(tick) and in_positive_sequence:
      if error_counter >= error_allowance:
        error_counter = 0
        in_positive_sequence = 0
        offsets.append((index - error_allowance) * single_tick_in_seconds)
      else:
        error_counter = error_counter + 1
    elif tick and in_positive_sequence and error_counter:
      error_counter = 0
  if in_positive_sequence:
    offsets.append((index) * single_tick_in_seconds)

  return onsets, offsets

def prepare_predicted_sequences_for_evaluation_and_evaluate(event_label, y_pred, I_test, training_data, events_data, return_prepared_data_flag = 0):
  references = []
  estimations = []

  for i, y_predicted in enumerate(y_pred):
    file_index = I_test[i]
    file_name = training_data.iloc[file_index]['filename']

    onsets, offsets = return_onsets_and_offsets_for_sequence(y_predicted, 10)

    for j, onset in enumerate(onsets):
      estimations.append({
          'event_label': event_label,
          'event_onset': onset,
          'event_offset': offsets[j],
          'file': file_name
      })

    reference_events = events_data[(events_data['filename'] == file_name) & (events_data['event_label'] == event_label)]
    for ref_event in np.asarray(reference_events):
      references.append({
          'event_label': event_label,
          'event_onset': ref_event[1],
          'event_offset': ref_event[2],
          'file': file_name
      })

  evaluate_and_print_results(references, estimations)

  if return_prepared_data_flag:
    return references, estimations


def evaluate_and_print_results(references, estimations):
  reference_event_list = dcase_util.containers.MetaDataContainer(
      references
  )

  estimated_event_list = dcase_util.containers.MetaDataContainer(
      estimations
  )

  segment_based_metrics = sed_eval.sound_event.SegmentBasedMetrics(
      event_label_list=reference_event_list.unique_event_labels,
      time_resolution=1.0
  )
  event_based_metrics = sed_eval.sound_event.EventBasedMetrics(
      event_label_list=reference_event_list.unique_event_labels,
      t_collar=0.250,
      evaluate_onset=True,
      evaluate_offset=True,
      percentage_of_length=0.5
  )

  for filename in reference_event_list.unique_files:
      reference_event_list_for_current_file = reference_event_list.filter(
          filename=filename
      )

      estimated_event_list_for_current_file = estimated_event_list.filter(
          filename=filename
      )

      segment_based_metrics.evaluate(
          reference_event_list=reference_event_list_for_current_file,
          estimated_event_list=estimated_event_list_for_current_file
      )

      event_based_metrics.evaluate(
          reference_event_list=reference_event_list_for_current_file,
          estimated_event_list=estimated_event_list_for_current_file
      )

  # Get only certain metrics
  overall_segment_based_metrics = segment_based_metrics.results_overall_metrics()
  print("Accuracy:", overall_segment_based_metrics['accuracy']['accuracy'])

  # Or print all metrics as reports
  print(segment_based_metrics)
  print(event_based_metrics)

     |████████████████████████████████| 2.1MB 39.8MB/s 
  Created wheel for sed-eval: filename=sed_eval-0.2.1-cp37-none-any.whl size=26124 sha256=6afbd0513794ee4c89e6df20f86f043774be97870f13130709c9ba53e607306b
  Stored in directory: /root/.cache/pip/wheels/3c/72/5d/5fc941f98c583ce9d8adee7f13e24b46459aeded4125f9c369
  Created wheel for dcase-util: filename=dcase_util-0.2.18-cp37-none-any.whl size=2147233 sha256=008df96271d30fb6afd450a36dd465e3b7a1a203b116fdefd1808243c006b797
  Stored in directory: /root/.cache/pip/wheels/d1/5d/9e/c94cd8bfb80541c73d549c734cafcd3de994dbb117a6485c81
Successfully built sed-eval dcase-util
